In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
def extrinsic_calib(points, us):
    points_number = np.shape(points)[0];
    ## o =  orignals
    ## c = correspondence
    o_centroid = np.mean(points, axis=0)
    c_centroid = np.mean(us, axis=0)
    
    o_centered = originals - np.tile(o_centroid, (points_number,1))
    c_centered = correspondence - np.tile(c_centroid, (points_number,1))
    
    o_norms = np.linalg.norm(o_centered, axis=1)
    c_norms = np.linalg.norm(c_centered, axis=1)                                           
    
    o_distance = np.mean(o_norms)
    c_distance = np.mean(c_norms)
                                          
    o_scale = np.sqrt(2)/o_distance
    c_scale = np.sqrt(2)/c_distance
    
    H_sim_prim = np.array([[c_scale,0      ,-c_centroid[0]*c_scale],
                          [0      ,c_scale,-c_centroid[1]*c_scale],
                          [0      ,0      ,1]])

    
    H_sim = np.array([[o_scale,0      ,-o_centroid[0]*o_scale],
                      [0      ,o_scale,-o_centroid[1]*o_scale],
                      [0      ,0      ,1]])
    
    points_norm = np.transpose(H_sim @ np.transpose(points))
    us_norm = np.transpose(H_sim_prim @ np.transpose(us))

    H_sim_prim = np.linalg.inv(H_sim_prim)
                                             
    A = np.zeros((points_number*2,9))

    zerot = np.array([0,0,0])

    for i in range(0, points_number):
        point = points_nomr[i, :]
        u = us_norm[i,:]
        aux = np.ndarray([0, point[0], u[1]*point[0], 0, -point[1], u[1]*point[1], 0, -1, u[1]])
        aux_one = np.ndarray([point[0], 0, -u[0]*point[0], point[1], 0, -u[0]*point[1], 1, 0, -u[0]])
        A[2*i,:] = aux
        A[2*i + 1,:] = aux_one
    (u,d,vt) = np.linalg.svd(A)
    H = np.reshape(vt[-1, :], (3,3))
    H = np.dot(H, H_sim)
    H = np.dot(H_sim_prim, H)
    return H

In [11]:
def centro_rotacion(Rs, Ts):
    identis = np.zeros(np.shape(Rs))
    for i in range(np.shape(Rs)[0]/3):
        identis[i*3,:] = np.ndarray([1, 0, 0])
        identis[i*3+1,:] = np.ndarray([0, 1, 0])
        identis[i*3+2,:] = np.ndarray([0, 0, 1])    
    sol = np.concatenate(Rs, identis)
    sol = np.delete(sol, 2, 1)
    
    (u,s,vt) = np.linalg.svd(sol)
    cs = np.dot(np.transpose(u), Ts)
    ys = np.zeros((5, 1))
    for i in range(5):
        if s[i, i] != 0:
            ys[i] = cs[i]/s[i,i]
    xs = np.dot(np.transpose(vt), ys)
    q = np.ndarray([xs[0], xs[1], 0])
    Q = xs[2:]
    return (q, Q)
        
    

In [10]:
np.identity(3)[1,0:]

array([0., 1., 0.])

In [7]:
aux = np.zeros((6,3))
np.delete(aux,1,1)

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])